# Модели со скрытыми факторами / матричные аппроксимации

<center><img src='imgs/matrix_factorization.png' width=600></center>

## SVD разложение / усеченное SVD
<center><img src='imgs/svd.png' width=600></center>

Замечу, что **матричная аппроксимация != матричное разложение**, хотя и очень близко по смыслу

<center><img src='imgs/mf2.png' width=700></center>

Для каждого пользователя и товара построим векторы $p_u\in \mathbb{R}^{k}$ и $q_i \in \mathbb{R}^{k}$ так, чтобы
$$ R_{ui} \approx p_u^\top q_i $$

* $p_u$ иногда получается интерпретировать как заинтересованность пользователя в некоторой категории товаров
* $q_i$ иногда получается интерпретировать как принадлежность товара к определенной категории

Кроме того, в полученном пространстве, можно считать похожесть пользователей и товаров

Будем оптимизировать следующий функционал
$$ \sum\limits_{u,i}(R_{ui}  - \langle p_u, q_i \rangle)^2 + \lambda \sum_u\| p_u \|^2 + \mu\sum_i\| q_i \|^2 \rightarrow \min\limits_{P, Q} $$
Решать задачу можно разными способами, один из стандартных - градиентный спуск (на каждом шаге случайно выбирая пару $(u,i)$:
$$ p_{uk} = p_{uk} + 2\alpha \left(q_{ik}(R_{ui} - \langle p_u, q_i \rangle) - \lambda p_{uk}\right)$$
$$ q_{ik} = q_{ik} + 2\alpha \left(p_{uk}(R_{ui} - \langle p_u, q_i \rangle) - \mu q_{ik}\right)$$

Иной способ - ALS, рассмотрим ниже

Результат обучения: матрицы P и Q. 

Их можно использовать как для получение предсказания для пары (u, i), так и для кластеризации пользователей / айтемов

In [ ]:
# Как соотносятся размеры матриц P, Q и R?

# Implicit/Explicit Feedback

Вид обратной связи может быть разным, выше описана постановка с случае явного фидбека.
Есть ее модификация на случай неявной обратной связи (один из подходов, называемый Weighted-Lamda-Regularization):

$$ \sum\limits_{u,i}(с_{ui}(r_{ui}  - \langle p_u, q_i \rangle)^2 + \lambda \sum_u\| p_u \|^2 + \mu\sum_i\| q_i \|^2 \rightarrow \min\limits_{P, Q} $$
$$ r_{ui} \in \{0, 1\}, \quad c_{ui} = 1 + \alpha R_{ui}$$
    

<center><img src='imgs/bpr.png' width=700></center>

# Важный частный случай: ALS-WR

Решаем оптимизационную задачу итерациями:
    1. Выбираем случайно приближение для матриц P и Q
    2. Находим наилучшую матрицу P при фиксированной Q
    3. Находим наилучшую матрицу Q при фиксированной матрице P
    4. Если не сошлись, то повторяем шаг 2

Быстро сходится, шаги 2/3 люди умеют делать эффективно, есть гарантия сходимости.


**Метод реализован много где, очень известна реализация на Spark, де-факто стандарт индустрии.**

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import scipy.sparse as sp
import pickle

In [ ]:
npz_file = np.load('./tmp/data.npz', allow_pickle=True)
train_ids = npz_file["train_ids"]
valid_ids = npz_file["valid_ids"]
NUM_ITEMS = 43038

In [ ]:
def make_coo_row(item_ids, num_items=NUM_ITEMS):
    idx = np.array(item_ids)
    values = np.ones(len(item_ids)).astype(np.float32)

    return sp.coo_matrix(
        (values, ([0] * len(idx), idx)), 
        shape=(1, num_items),
    )

In [ ]:
rows = []
for user_history in tqdm(train_ids):
    rows.append(make_coo_row(user_history))
X_sparse = sp.vstack(rows).tocsr()

In [ ]:
import implicit
model = implicit.als.AlternatingLeastSquares(factors=16, regularization=0.001, iterations=8)
model.fit(X_sparse.T)

In [ ]:
# Какой будет размер матриц P и Q внутри модели?
# Как он (размер) зависит от числа итераций?

In [ ]:
model.user_factors.shape, model.item_factors.shape

In [ ]:
model.recommend(10, X_sparse, N=10, filter_already_liked_items=False)

In [ ]:
from src.metrics import normalized_average_precision

m_ap = []
for i, gt_ids in tqdm(enumerate(valid_ids)):
    rec_raw = model.recommend(i, X_sparse, N=30, filter_already_liked_items=False)
    rec_ids = [x[0] for x in rec_raw]
    m_ap.append(normalized_average_precision(gt_ids, rec_ids, k=30))
print(np.mean(m_ap))

# Эффекты скорости

In [ ]:
%%timeit
_ = model.recommend(10, X_sparse, N=1, filter_already_liked_items=False)

In [ ]:
%%timeit
_ = model.recommend(10, X_sparse, N=100, filter_already_liked_items=False)

In [ ]:
%%timeit
_ = model.recommend(10, X_sparse, N=1000, filter_already_liked_items=False)

In [ ]:
%%timeit
_ = model.recommend(10, X_sparse, N=10000, filter_already_liked_items=False)

In [ ]:
selected_items = list(range(10))

In [ ]:
%%timeit
_ = model.rank_items(10, X_sparse, selected_items)

# Количество рекомендаций

In [ ]:
len(model.recommend(10, X_sparse, N=30000, filter_already_liked_items=False))

### Важная особенность ALS: возможность пересчет профиля пользователя "на лету"

In [ ]:
item_ids = train_ids[10] #история пользователя

model.recommend(10, X_sparse, N=5, filter_already_liked_items=True)

In [ ]:
row = make_coo_row(item_ids).tocsr()
model.recommend(0, row, N=5, filter_already_liked_items=True, recalculate_user=True)

In [ ]:
# Есть ли различия? Почему?

In [ ]:
new_history = list(item_ids) + [41915]
row = make_coo_row(new_history).tocsr()
model.recommend(0, row, N=5, filter_already_liked_items=True, recalculate_user=True)

In [ ]:
# Что можно твикать?
# Как можно использовать факторы?
# Какую оптимизационную задачу решаем? Это классификация/регрессия/что-то еще?

# BPR: Bayesian personalized ranking

<center><img src='imgs/bpr.png' width=500></center>

<center><img src='imgs/bpr-2.png' width=450></center>

<center><img src='imgs/bprf.png' width=300></center>

In [ ]:
model = implicit.bpr.BayesianPersonalizedRanking(factors=16)
model.fit(X_sparse.T)

In [ ]:
from src.metrics import normalized_average_precision

m_ap = []
for i, gt_ids in tqdm(enumerate(valid_ids)):
    rec_raw = model.recommend(i, X_sparse, N=30, filter_already_liked_items=False)
    rec_ids = [x[0] for x in rec_raw]
    m_ap.append(normalized_average_precision(gt_ids, rec_ids, k=30))
print(np.mean(m_ap))

# Важность параметров 
### (как и во многих других методах, основанных на градиентном спуске)

# Время выполнения

In [ ]:
%%timeit
_ = model.recommend(10, X_sparse, N=1, filter_already_liked_items=False)

In [ ]:
%%timeit
_ = model.recommend(10, X_sparse, N=100, filter_already_liked_items=False)

In [ ]:
%%timeit
_ = model.recommend(10, X_sparse, N=1000, filter_already_liked_items=False)

In [ ]:
%%timeit
_ = model.recommend(10, X_sparse, N=10000, filter_already_liked_items=False)

In [ ]:
# Как время исполнения соотносится с ALS?